In [1]:
%pwd

'C:\\NN-P4\\python-notebooks'

###Imports and Read CSV

In [2]:
import json
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from datetime import datetime
from keras.callbacks import History # to get train accuracy
history = History()

In [3]:
#nPrintML creates a copy of dataset in pkl picles format. Convert to CSV 1st.
# Read the PKL
# df_train = pd.read_pickle('../datasets/nprint-raw/stream-video/X.pkl')
# df_test = pd.read_pickle('../datasets/nprint-raw/stream-video/X_val.pkl')
# label_train = pd.read_pickle('../datasets/nprint-raw/stream-video/y.pkl')
# label_test = pd.read_pickle('../datasets/nprint-raw/stream-video/y_val.pkl')
# # Save a copy as CSV
# df_train.to_csv('../datasets/nprint-raw/stream-video/X.csv')
# df_test.to_csv('../datasets/nprint-raw/stream-video/X_val.csv')
# label_train.to_csv('../datasets/nprint-raw/stream-video/y.csv')
# label_test.to_csv('../datasets/nprint-raw/stream-video/y_val.csv')
# Read CSV
df_train = pd.read_csv('../datasets/nprint-raw/stream-video/X.csv')
df_test = pd.read_csv('../datasets/nprint-raw/stream-video/X_val.csv')
label_train = pd.read_csv('../datasets/nprint-raw/stream-video/y.csv')
label_test = pd.read_csv('../datasets/nprint-raw/stream-video/y_val.csv')
#Drop colum with pcap sequence number
df_train.drop('pcap', axis=1, inplace=True)
df_test.drop('pcap', axis=1, inplace=True)
label_train.drop('pcap', axis=1, inplace=True)
label_test.drop('pcap', axis=1, inplace=True)
#Rename Unnamed Collumn
label_train.rename(columns={'0': 'label'}, inplace=True)
label_test.rename(columns={'0': 'label'}, inplace=True)
#Read file with ordered list of attributes ranked by the autoML feature importance algorithm
features_rankings_df = pd.read_csv('../datasets/nprint-raw/stream-video/feature-importance.csv')
features_rankings_df.rename(columns={'0': 'label'}, inplace=True)

In [4]:
display(df_train)
display(df_test)
display(label_train)
display(label_test)
print(label_train["label"].value_counts())
print(label_test["label"].value_counts())
print(f"df_train shape: {df_train.shape} \n columns: {df_train.columns.values}")
print(f"df_test shape: {df_test.shape} \n columns: {df_test.columns.values}")
display(features_rankings_df.head(40))

,pkt_0_ipv4_tl_9,pkt_0_ipv4_tl_10,pkt_0_ipv4_tl_11,pkt_0_ipv4_tl_12,pkt_0_ipv4_tl_13,pkt_0_ipv4_id_0,pkt_0_ipv4_id_1,pkt_0_ipv4_id_2,pkt_0_ipv4_id_3,pkt_0_ipv4_id_4,...,pkt_9_tcp_opt_182,pkt_9_tcp_opt_183,pkt_9_tcp_opt_184,pkt_9_tcp_opt_185,pkt_9_tcp_opt_186,pkt_9_tcp_opt_187,pkt_9_tcp_opt_188,pkt_9_tcp_opt_189,pkt_9_tcp_opt_190,pkt_9_tcp_opt_191
0,1,0,0,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,0,1,0,0,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,0,1,1,1,1,1,1,0,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,0,1,1,1,1,0,0,1,1,1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,0,1,1,1,1,1,1,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6592,0,1,1,1,1,1,1,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6593,1,0,0,0,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
6594,1,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
6595,0,1,1,1,1,0,1,1,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1


,pkt_0_ipv4_tl_9,pkt_0_ipv4_tl_10,pkt_0_ipv4_tl_11,pkt_0_ipv4_tl_12,pkt_0_ipv4_tl_13,pkt_0_ipv4_id_0,pkt_0_ipv4_id_1,pkt_0_ipv4_id_2,pkt_0_ipv4_id_3,pkt_0_ipv4_id_4,...,pkt_9_tcp_opt_182,pkt_9_tcp_opt_183,pkt_9_tcp_opt_184,pkt_9_tcp_opt_185,pkt_9_tcp_opt_186,pkt_9_tcp_opt_187,pkt_9_tcp_opt_188,pkt_9_tcp_opt_189,pkt_9_tcp_opt_190,pkt_9_tcp_opt_191
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,1,1,1,1,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,0,1,1,0,1,0,0,0,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,0,1,1,1,1,1,0,0,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,0,1,1,0,1,0,0,0,1,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
729,0,1,1,1,1,0,1,0,0,0,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
730,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
731,1,0,0,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0


,label
0,3
1,1
2,3
3,2
4,0
...,...
6592,3
6593,1
6594,3
6595,1


,label
0,1
1,3
2,3
3,1
4,2
...,...
728,1
729,3
730,2
731,3


label
3    2944
1    2028
2    1019
0     606
Name: count, dtype: int64
label
3    327
1    226
2    113
0     67
Name: count, dtype: int64
df_train shape: (6597, 4491) 
 columns: ['pkt_0_ipv4_tl_9' 'pkt_0_ipv4_tl_10' 'pkt_0_ipv4_tl_11' ...
 'pkt_9_tcp_opt_189' 'pkt_9_tcp_opt_190' 'pkt_9_tcp_opt_191']
df_test shape: (733, 4491) 
 columns: ['pkt_0_ipv4_tl_9' 'pkt_0_ipv4_tl_10' 'pkt_0_ipv4_tl_11' ...
 'pkt_9_tcp_opt_189' 'pkt_9_tcp_opt_190' 'pkt_9_tcp_opt_191']


,Unnamed: 0,importance,stddev,p_value,n,p99_high,p99_low
0,pkt_3_ipv4_src_23,0.008186,0.000910,1.986209e-10,10,0.009120,0.007251
1,pkt_0_ipv4_src_25,0.006412,0.001123,1.118015e-08,10,0.007566,0.005258
2,pkt_9_ipv4_dst_5,0.005457,0.001286,1.480476e-07,10,0.006779,0.004135
3,pkt_2_rts,0.005457,0.001286,1.480476e-07,10,0.006779,0.004135
4,pkt_9_ipv4_dst_6,0.004229,0.001977,4.113168e-05,10,0.006261,0.002197
5,pkt_9_ipv4_dst_0,0.004093,0.002034,6.535170e-05,10,0.006183,0.002003
6,pkt_4_rts,0.003820,0.002113,1.443004e-04,10,0.005992,0.001648
7,pkt_3_rts,0.003547,0.002246,3.728090e-04,10,0.005856,0.001239
8,pkt_7_ipv4_dst_27,0.003001,0.001908,3.823591e-04,10,0.004962,0.001041
9,pkt_6_tcp_opt_60,0.002865,0.001357,4.539357e-05,10,0.004259,0.001471


In [5]:
#make a list with only the best features

feature_list = features_rankings_df['Unnamed: 0'].tolist()
feature_list = feature_list[:32]
print(feature_list)

#select a sub-dataframe with only the 32 top attrs rate by the AUTOGLUON feature importance algorithm
df_train = df_train[feature_list]
df_test = df_test[feature_list]

display(df_train)
display(df_test)

['pkt_3_ipv4_src_23', 'pkt_0_ipv4_src_25', 'pkt_9_ipv4_dst_5', 'pkt_2_rts', 'pkt_9_ipv4_dst_6', 'pkt_9_ipv4_dst_0', 'pkt_4_rts', 'pkt_3_rts', 'pkt_7_ipv4_dst_27', 'pkt_6_tcp_opt_60', 'pkt_6_ipv4_dst_4', 'pkt_0_ipv4_src_18', 'pkt_7_rts', 'pkt_9_ipv4_src_28', 'pkt_4_tcp_opt_60', 'pkt_0_tcp_opt_60', 'pkt_9_ipv4_dst_4', 'pkt_8_rts', 'pkt_4_ipv4_dst_1', 'pkt_8_ipv4_src_28', 'pkt_9_ipv4_dst_3', 'pkt_5_rts', 'pkt_7_ipv4_dst_11', 'pkt_1_rts', 'pkt_9_ipv4_src_21', 'pkt_7_ipv4_dst_9', 'pkt_9_ipv4_dst_13', 'pkt_4_ipv4_dst_12', 'pkt_7_ipv4_dst_0', 'pkt_6_rts', 'pkt_2_ipv4_dst_19', 'pkt_3_ipv4_dst_7']


,pkt_3_ipv4_src_23,pkt_0_ipv4_src_25,pkt_9_ipv4_dst_5,pkt_2_rts,pkt_9_ipv4_dst_6,pkt_9_ipv4_dst_0,pkt_4_rts,pkt_3_rts,pkt_7_ipv4_dst_27,pkt_6_tcp_opt_60,...,pkt_7_ipv4_dst_11,pkt_1_rts,pkt_9_ipv4_src_21,pkt_7_ipv4_dst_9,pkt_9_ipv4_dst_13,pkt_4_ipv4_dst_12,pkt_7_ipv4_dst_0,pkt_6_rts,pkt_2_ipv4_dst_19,pkt_3_ipv4_dst_7
0,1,0,0,1362319,0,0,71,132,0,0,...,1,296,0,0,0,1,0,69,0,0
1,1,0,1,285,1,0,926,104160,0,1,...,0,4390256,0,0,0,0,0,634,0,0
2,1,0,1,326,0,0,219,205,1,1,...,0,1352462,0,1,0,1,0,121,1,0
3,1,0,0,33,0,1,14186,20,0,1,...,1,43,1,1,0,0,1,38,1,1
4,1,1,0,1190046,0,1,59,116,1,1,...,1,244,0,1,0,1,1,55,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6592,1,0,0,241610,1,1,7998,621382,0,1,...,1,11881,0,0,1,1,0,44405,1,0
6593,1,0,1,214,1,0,142,1531520,0,0,...,0,320,0,0,0,0,0,78,0,0
6594,1,0,1,155,1,1,445725,1395299,1,0,...,1,162,0,0,0,1,0,505802,1,0
6595,1,0,1,27,1,0,24,24,0,1,...,0,150058,0,0,0,0,0,20,1,0


,pkt_3_ipv4_src_23,pkt_0_ipv4_src_25,pkt_9_ipv4_dst_5,pkt_2_rts,pkt_9_ipv4_dst_6,pkt_9_ipv4_dst_0,pkt_4_rts,pkt_3_rts,pkt_7_ipv4_dst_27,pkt_6_tcp_opt_60,...,pkt_7_ipv4_dst_11,pkt_1_rts,pkt_9_ipv4_src_21,pkt_7_ipv4_dst_9,pkt_9_ipv4_dst_13,pkt_4_ipv4_dst_12,pkt_7_ipv4_dst_0,pkt_6_rts,pkt_2_ipv4_dst_19,pkt_3_ipv4_dst_7
0,1,0,1,151,1,0,53,1431882,0,0,...,0,370,0,0,0,0,0,314,1,0
1,1,0,0,2076744,0,0,3786926,119549565,1,0,...,1,1197,0,0,0,1,0,31323367,0,0
2,1,0,0,967187,0,0,150,196629,0,1,...,1,7963,0,0,1,1,0,58844,0,0
3,1,0,1,94780,1,0,319,274,1,1,...,0,272,0,0,0,0,0,704,1,0
4,1,0,1,40,1,0,7267,36,1,1,...,1,72,1,0,1,0,1,35,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,0,1,0,421237,0,0,47848,121529,0,0,...,1,119321,1,0,0,1,0,183,0,0
729,1,1,0,72,0,0,3044,12488,0,1,...,1,3547,0,0,0,1,0,56,1,0
730,1,0,0,1069169,1,1,170952,104,0,0,...,0,284,0,1,0,1,0,73,0,1
731,1,0,1,148,0,0,1178443,1453628,0,0,...,1,167,0,0,0,1,0,277150,1,0


In [6]:
INTEGER = 8

mini = []
maxi = []
column_list = ["pkt_1_rts", "pkt_2_rts", "pkt_3_rts", "pkt_4_rts", "pkt_5_rts", "pkt_6_rts", "pkt_7_rts", "pkt_8_rts"]
for column_name in column_list:
    mini.append(df_test[column_name].min())
    print(mini)    
    maxi.append(df_test[column_name].max())
    print(maxi)

maximo = max(maxi)
minimo = min(mini)
print(maximo)


display(df_test)
for column_name in column_list:
    aux = []
    for x in df_test[column_name]:
        normalized_value = (x - minimo) / (maximo - minimo) #norm values betwen 0 and 1
        scaled_value = normalized_value * (2**INTEGER - 1) #deslocate values left to make use of the INTEGER bits
        aux.append(scaled_value)
    df_test[column_name]=aux    
display(df_test)

mini = []
maxi = []
for column_name in column_list:
    mini.append(df_train[column_name].min())
    print(mini)    
    maxi.append(df_train[column_name].max())
    print(maxi)

maximo = max(maxi)
minimo = min(mini)
print(maximo)


display(df_train)
for column_name in column_list:
    aux = []
    for x in df_train[column_name]:
        normalized_value = (x - minimo) / (maximo - minimo)
        scaled_value = normalized_value * (2**INTEGER - 1)
        aux.append(scaled_value)
    df_train[column_name]=aux
    
display(df_train)

# for column_name in column_list:
#     for x in df_test[column_name]:
#         normalized_value = (x - minimo) / (maximo - minimo)
#         scaled_value = normalized_value * (2**16 - 1)
#         df_new.loc[x, column_name] = scaled_value
    
df_train.to_csv('../datasets/nprint-raw/stream-video-fixed/X.csv')
df_test.to_csv('../datasets/nprint-raw/stream-video-fixed/X_val.csv')
label_train.to_csv('../datasets/nprint-raw/stream-video-fixed/y.csv')
label_test.to_csv('../datasets/nprint-raw/stream-video-fixed/y_val.csv')


[0]
[358648184]
[0, 0]
[358648184, 356388935]
[0, 0, -1]
[358648184, 356388935, 358855971]
[0, 0, -1, -1]
[358648184, 356388935, 358855971, 361459473]
[0, 0, -1, -1, -1]
[358648184, 356388935, 358855971, 361459473, 361145967]
[0, 0, -1, -1, -1, -1]
[358648184, 356388935, 358855971, 361459473, 361145967, 357808985]
[0, 0, -1, -1, -1, -1, -1]
[358648184, 356388935, 358855971, 361459473, 361145967, 357808985, 361397597]
[0, 0, -1, -1, -1, -1, -1, -1]
[358648184, 356388935, 358855971, 361459473, 361145967, 357808985, 361397597, 364156899]
364156899


,pkt_3_ipv4_src_23,pkt_0_ipv4_src_25,pkt_9_ipv4_dst_5,pkt_2_rts,pkt_9_ipv4_dst_6,pkt_9_ipv4_dst_0,pkt_4_rts,pkt_3_rts,pkt_7_ipv4_dst_27,pkt_6_tcp_opt_60,...,pkt_7_ipv4_dst_11,pkt_1_rts,pkt_9_ipv4_src_21,pkt_7_ipv4_dst_9,pkt_9_ipv4_dst_13,pkt_4_ipv4_dst_12,pkt_7_ipv4_dst_0,pkt_6_rts,pkt_2_ipv4_dst_19,pkt_3_ipv4_dst_7
0,1,0,1,151,1,0,53,1431882,0,0,...,0,370,0,0,0,0,0,314,1,0
1,1,0,0,2076744,0,0,3786926,119549565,1,0,...,1,1197,0,0,0,1,0,31323367,0,0
2,1,0,0,967187,0,0,150,196629,0,1,...,1,7963,0,0,1,1,0,58844,0,0
3,1,0,1,94780,1,0,319,274,1,1,...,0,272,0,0,0,0,0,704,1,0
4,1,0,1,40,1,0,7267,36,1,1,...,1,72,1,0,1,0,1,35,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,0,1,0,421237,0,0,47848,121529,0,0,...,1,119321,1,0,0,1,0,183,0,0
729,1,1,0,72,0,0,3044,12488,0,1,...,1,3547,0,0,0,1,0,56,1,0
730,1,0,0,1069169,1,1,170952,104,0,0,...,0,284,0,1,0,1,0,73,0,1
731,1,0,1,148,0,0,1178443,1453628,0,0,...,1,167,0,0,0,1,0,277150,1,0


,pkt_3_ipv4_src_23,pkt_0_ipv4_src_25,pkt_9_ipv4_dst_5,pkt_2_rts,pkt_9_ipv4_dst_6,pkt_9_ipv4_dst_0,pkt_4_rts,pkt_3_rts,pkt_7_ipv4_dst_27,pkt_6_tcp_opt_60,...,pkt_7_ipv4_dst_11,pkt_1_rts,pkt_9_ipv4_src_21,pkt_7_ipv4_dst_9,pkt_9_ipv4_dst_13,pkt_4_ipv4_dst_12,pkt_7_ipv4_dst_0,pkt_6_rts,pkt_2_ipv4_dst_19,pkt_3_ipv4_dst_7
0,1,0,1,0.000106,1,0,0.000038,1.002673,0,0,...,0,0.000260,0,0,0,0,0,0.000221,1,0
1,1,0,0,1.454236,0,0,2.651787,83.714298,1,0,...,1,0.000839,0,0,0,1,0,21.934114,0,0
2,1,0,0,0.677271,0,0,0.000106,0.137690,0,1,...,1,0.005577,0,0,1,1,0,0.041206,0,0
3,1,0,1,0.066370,1,0,0.000224,0.000193,1,1,...,0,0.000191,0,0,0,0,0,0.000494,1,0
4,1,0,1,0.000029,1,0,0.005089,0.000026,1,1,...,1,0.000051,1,0,1,0,1,0.000025,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
728,0,1,0,0.294971,0,0,0.033506,0.085101,0,0,...,1,0.083555,1,0,0,1,0,0.000129,0,0
729,1,1,0,0.000051,0,0,0.002132,0.008745,0,1,...,1,0.002484,0,0,0,1,0,0.000040,1,0
730,1,0,0,0.748684,1,1,0.119709,0.000074,0,0,...,0,0.000200,0,1,0,1,0,0.000052,0,1
731,1,0,1,0.000104,0,0,0.825203,1.017900,0,0,...,1,0.000118,0,0,0,1,0,0.194074,1,0


[0]
[359235600]
[0, 0]
[359235600, 359517995]
[0, 0, -1]
[359235600, 359517995, 359397735]
[0, 0, -1, -1]
[359235600, 359517995, 359397735, 358981693]
[0, 0, -1, -1, -1]
[359235600, 359517995, 359397735, 358981693, 361410870]
[0, 0, -1, -1, -1, -1]
[359235600, 359517995, 359397735, 358981693, 361410870, 363893498]
[0, 0, -1, -1, -1, -1, -1]
[359235600, 359517995, 359397735, 358981693, 361410870, 363893498, 361459333]
[0, 0, -1, -1, -1, -1, -1, -1]
[359235600, 359517995, 359397735, 358981693, 361410870, 363893498, 361459333, 362452485]
363893498


,pkt_3_ipv4_src_23,pkt_0_ipv4_src_25,pkt_9_ipv4_dst_5,pkt_2_rts,pkt_9_ipv4_dst_6,pkt_9_ipv4_dst_0,pkt_4_rts,pkt_3_rts,pkt_7_ipv4_dst_27,pkt_6_tcp_opt_60,...,pkt_7_ipv4_dst_11,pkt_1_rts,pkt_9_ipv4_src_21,pkt_7_ipv4_dst_9,pkt_9_ipv4_dst_13,pkt_4_ipv4_dst_12,pkt_7_ipv4_dst_0,pkt_6_rts,pkt_2_ipv4_dst_19,pkt_3_ipv4_dst_7
0,1,0,0,1362319,0,0,71,132,0,0,...,1,296,0,0,0,1,0,69,0,0
1,1,0,1,285,1,0,926,104160,0,1,...,0,4390256,0,0,0,0,0,634,0,0
2,1,0,1,326,0,0,219,205,1,1,...,0,1352462,0,1,0,1,0,121,1,0
3,1,0,0,33,0,1,14186,20,0,1,...,1,43,1,1,0,0,1,38,1,1
4,1,1,0,1190046,0,1,59,116,1,1,...,1,244,0,1,0,1,1,55,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6592,1,0,0,241610,1,1,7998,621382,0,1,...,1,11881,0,0,1,1,0,44405,1,0
6593,1,0,1,214,1,0,142,1531520,0,0,...,0,320,0,0,0,0,0,78,0,0
6594,1,0,1,155,1,1,445725,1395299,1,0,...,1,162,0,0,0,1,0,505802,1,0
6595,1,0,1,27,1,0,24,24,0,1,...,0,150058,0,0,0,0,0,20,1,0


,pkt_3_ipv4_src_23,pkt_0_ipv4_src_25,pkt_9_ipv4_dst_5,pkt_2_rts,pkt_9_ipv4_dst_6,pkt_9_ipv4_dst_0,pkt_4_rts,pkt_3_rts,pkt_7_ipv4_dst_27,pkt_6_tcp_opt_60,...,pkt_7_ipv4_dst_11,pkt_1_rts,pkt_9_ipv4_src_21,pkt_7_ipv4_dst_9,pkt_9_ipv4_dst_13,pkt_4_ipv4_dst_12,pkt_7_ipv4_dst_0,pkt_6_rts,pkt_2_ipv4_dst_19,pkt_3_ipv4_dst_7
0,1,0,0,0.954652,0,0,0.000050,0.000093,0,0,...,1,0.000208,0,0,0,1,0,0.000049,0,0
1,1,0,1,0.000200,1,0,0.000650,0.072991,0,1,...,0,3.076492,0,0,0,0,0,0.000445,0,0
2,1,0,1,0.000229,0,0,0.000154,0.000144,1,1,...,0,0.947745,0,1,0,1,0,0.000085,1,0
3,1,0,0,0.000024,0,1,0.009942,0.000015,0,1,...,1,0.000031,1,1,0,0,1,0.000027,1,1
4,1,1,0,0.833931,0,1,0.000042,0.000082,1,1,...,1,0.000172,0,1,0,1,1,0.000039,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6592,1,0,0,0.169310,1,1,0.005605,0.435437,0,1,...,1,0.008326,0,0,1,1,0,0.031118,1,0
6593,1,0,1,0.000151,1,0,0.000100,1.073220,0,0,...,0,0.000225,0,0,0,0,0,0.000055,0,0
6594,1,0,1,0.000109,1,1,0.312344,0.977763,1,0,...,1,0.000114,0,0,0,1,0,0.354444,1,0
6595,1,0,1,0.000020,1,0,0.000018,0.000018,0,1,...,0,0.105155,0,0,0,0,0,0.000015,1,0


In [7]:
# Model Parameters
number_of_attributes = 8
hidden_layer_nodes = 64
output_layer_nodes = 4

# Traing Parameters
EPOCHS = 500
BATCH_SIZE = 500

# Run the NN Training multiple times to get a nice mean score of accuracy results
NUMBER_OF_NN_REPETITIONS = 1

def get_basic_model():
  model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(hidden_layer_nodes, activation='relu'),
    tf.keras.layers.Dense(output_layer_nodes)
  ])
  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [8]:
# Convert int values to TF expect values (float)
df_train = np.asarray(df_train).astype(np.float32)
df_test = np.asarray(df_test).astype(np.float32)
# Converts pandas dataframe to tensorflow object
df_train = tf.convert_to_tensor(df_train)
# Set the 
normalizer = tf.keras.layers.Normalization(axis=-1)
# Normalize the data
normalizer.adapt(df_train)
#normalizer.adapt(numeric_features_test)

In [9]:
# Execute Trainning and Testing
#model = get_basic_model()
train_accuracy = []
test_accuracy = []

for i in range(NUMBER_OF_NN_REPETITIONS):
  print(f"NN REPETITION {i+1}")
  model = get_basic_model()
  history = model.fit(df_train, label_train, epochs=EPOCHS, verbose=0, batch_size=BATCH_SIZE, callbacks=[history])
  one_train_accuracy = history.history['accuracy']

  test_loss, one_test_accuracy =  model.evaluate(df_test,  label_test, verbose=1, batch_size=BATCH_SIZE)
  train_accuracy.append(one_train_accuracy[-1])
  test_accuracy.append(one_test_accuracy)
  print(f"{i+1}: train_accuracy:{train_accuracy} test_accuracy:{test_accuracy}")
  tf_predictions_probabilities = model.predict(df_test)
  #tf_loss, tf_acc =  model.evaluate(numeric_features_test,  label_test, verbose=2, batch_size=BATCH_SIZE)

  #Confusion Matrix
  tf_predictions = []
  for i,x in enumerate(tf_predictions_probabilities):
    #print("i:",i,"x_max:",x.max(),"x:",x)
    j_max = x.argmax()
    tf_predictions.append(j_max)
  conf_m = tf.math.confusion_matrix(label_test,tf_predictions)
  print(conf_m)
  
  # Precision, Recall, F1 Score Computation
  label_test = np.array(label_test)  # True labels
  tf_predictions = np.array(tf_predictions)  # Predicted labels
  # Number of classes
  num_classes = output_layer_nodes  
  # Initialize metrics
  precision_list = []
  recall_list = []
  f1_list = []
  auc_list = []    
  # Calculate metrics for each class
  for i in range(num_classes):
      # Binary labels for class i (one-vs-rest)
      true_binary = (label_test == i)  # This is a boolean series now
      pred_binary = (tf_predictions == i)
      
      # True Positives, False Positives, False Negatives
      TP = np.sum(true_binary & pred_binary)
      FP = np.sum(~true_binary & pred_binary)
      FN = np.sum(true_binary & ~pred_binary)
      
      # Precision, Recall, F1 for class i
      precision = TP / (TP + FP) if (TP + FP) > 0 else 0
      recall = TP / (TP + FN) if (TP + FN) > 0 else 0
      f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
      
      # Store the metrics
      precision_list.append(precision)
      recall_list.append(recall)
      f1_list.append(f1)
            
  print(f'precision_list: {precision_list}, recall_list: {recall_list}, f1_list: {f1_list}')
  # Calculate macro-averaged metrics
  precision_macro = np.sum(precision_list) #ORIGINAL FORMULA WAS MEAN
  recall_macro = np.sum(recall_list)
  f1_macro = np.sum(f1_list)
  print(f'precision_macro: {precision_macro}, recall_macro: {recall_macro}, f1_macro: {f1_macro}')

#After n repetitions of training and testing, calculate statistics
train_accuracy_mean = sum(train_accuracy) / len(train_accuracy)
train_max = max(train_accuracy)
train_min = min(train_accuracy)
train_std = np.std(train_accuracy)
test_accuracy_mean = sum(test_accuracy) / len(test_accuracy)
test_max = max(test_accuracy)
test_min = min(test_accuracy)
test_std = np.std(test_accuracy)

print(f'accuracies_train: {train_accuracy}, accuracies_test: {test_accuracy}')
print(f'train_accuracy_mean: {train_accuracy_mean*100:.2f}, train_max: {train_max*100:.2f}, train_min: {train_min*100:.2f}, train_std: {train_std*100:.2f} \n test_accuracy_mean: {test_accuracy_mean*100:.2f}, test_max: {test_max*100:.2f}, test_min: {test_min*100:.2f}, test_std: {test_std*100:.2f}')



NN REPETITION 1
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9590 - loss: 0.1380 
1: train_accuracy:[0.9795361757278442] test_accuracy:[0.9604365825653076]
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
tf.Tensor(
[[ 62   2   3   0]
 [  6 213   3   4]
 [  3   1 109   0]
 [  1   5   1 320]], shape=(4, 4), dtype=int32)
precision_list: [0.09140518417462483, 0.3083219645293315, 0.15416098226466576, 0.4461118690313779], recall_list: [0.09822646657571624, 0.3015006821282401, 0.1582537517053206, 0.44201909959072305], f1_list: [0.09469314043989911, 0.30487317297978644, 0.1561805584515391, 0.4440560539667172]
precision_macro: 1.0, recall_macro: 1.0, f1_macro: 0.9998029258379418
accuracies_train: [0.9795361757278442], accuracies_test: [0.9604365825653076]
train_accuracy_mean: 97.95, train_max: 97.95, train_min: 97.95, train_std: 0.00 
 test_accuracy_mean: 96.04, test_max: 96.04, test_min: 96.04, test_std: 0.00


In [10]:
# Date for report file
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y-%H-%M-%S")
print(dt_string)
# Report file content
report = {
    "datetime": dt_string,
    "training_dataset_shape":  str(df_train.shape),
    "test_dataset_shape": str(df_test.shape),
    "number_of_attributes": number_of_attributes,
    "n_epoch":EPOCHS,
    "accuracy_train": train_accuracy_mean,
    "train_max": train_max,
    "train_min": train_min,
    "train_std": train_std,
    "accuracy_test": test_accuracy_mean,
    "test_max": test_max,
    "test_min": test_min,
    "test_std": test_std,
    "accuracies_train": train_accuracy,
    "accuracies_test": test_accuracy
}

# Writes into the file
os.chdir('../tf-model-testing')
print(os.getcwd())

title_parameters_save = f"nn-nprint-video-stream-model-evaluation-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

title_parameters_save = f"nn-nprint-video-stream-model-evaluation-{number_of_attributes}x{hidden_layer_nodes}x{output_layer_nodes}-{dt_string}.json"
with open(title_parameters_save, "w") as f:
  json.dump(report, f)

09-09-2024-15-09-51
C:\NN-P4\tf-model-testing
